# Kernel Semantik dengan Integrasi Server MCP OpenBnB

Notebook ini menunjukkan cara menggunakan Kernel Semantik dengan server MCP OpenBnB yang sebenarnya untuk mencari akomodasi Airbnb nyata menggunakan MCPStdioPlugin. Untuk akses LLM, ini menggunakan Azure AI Foundry. Untuk mengatur variabel lingkungan Anda, Anda dapat mengikuti [Pelajaran Pengaturan](/00-course-setup/README.md)


## Impor Paket yang Dibutuhkan


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Membuat Koneksi Plugin MCP

Kita akan terhubung ke [server MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) menggunakan MCPStdioPlugin. Server ini menyediakan fungsi pencarian Airbnb melalui paket @openbnb/mcp-server-airbnb.


## Membuat Klien

Dalam contoh ini, kita akan menggunakan Azure AI Foundry untuk akses LLM kita. Pastikan variabel lingkungan Anda telah diatur dengan benar.


## Konfigurasi Lingkungan

Konfigurasikan pengaturan Azure OpenAI. Pastikan Anda telah menetapkan variabel lingkungan berikut:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Memahami Integrasi OpenBnB MCP

Notebook ini terhubung ke **server OpenBnB MCP asli** yang menyediakan fungsi pencarian Airbnb secara nyata.

### Cara kerjanya:

1. **MCPStdioPlugin**: Menggunakan komunikasi input/output standar dengan server MCP
2. **Paket NPM Asli**: Mengunduh dan menjalankan `@openbnb/mcp-server-airbnb` melalui npx
3. **Data Langsung**: Mengembalikan data properti Airbnb nyata dari API mereka
4. **Penemuan Fungsi**: Agen secara otomatis menemukan fungsi yang tersedia dari server MCP

### Fungsi yang Tersedia:

Server OpenBnB MCP biasanya menyediakan:
- **search_listings** - Mencari properti Airbnb berdasarkan lokasi dan kriteria
- **get_listing_details** - Mendapatkan informasi detail tentang properti tertentu
- **check_availability** - Memeriksa ketersediaan untuk tanggal tertentu
- **get_reviews** - Mengambil ulasan untuk properti
- **get_host_info** - Mendapatkan informasi tentang pemilik properti

### Prasyarat:

- **Node.js** terinstal di sistem Anda
- **Koneksi internet** untuk mengunduh paket server MCP
- **NPX** tersedia (sudah termasuk dengan Node.js)

### Menguji Koneksi:

Anda dapat menguji server MCP secara manual dengan menjalankan:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Ini akan mengunduh dan memulai server OpenBnB MCP, yang kemudian akan terhubung dengan Semantic Kernel untuk mendapatkan data Airbnb secara nyata.


## Menjalankan Agen dengan Server OpenBnB MCP

Sekarang kita akan menjalankan Agen AI yang terhubung ke server OpenBnB MCP untuk mencari akomodasi Airbnb asli di Stockholm untuk 2 dewasa dan 1 anak. Anda dapat mengubah daftar `user_inputs` untuk memodifikasi kriteria pencarian.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

Ringkasan  
Selamat! Anda telah berhasil membangun agen AI yang terintegrasi dengan pencarian akomodasi dunia nyata menggunakan Model Context Protocol (MCP):

Teknologi yang Digunakan:  
- Semantic Kernel - Untuk membangun agen cerdas dengan Azure OpenAI  
- Azure AI Foundry - Untuk kemampuan LLM dan penyelesaian percakapan  
- MCP (Model Context Protocol) - Untuk integrasi alat yang terstandarisasi  
- OpenBnB MCP Server - Untuk fungsi pencarian Airbnb yang nyata  
- Node.js/NPX - Untuk menjalankan server MCP eksternal  

Apa yang Telah Anda Pelajari:  
- Integrasi MCP: Menghubungkan agen Semantic Kernel ke server MCP eksternal  
- Akses Data Real-time: Mencari properti Airbnb secara langsung melalui API live  
- Komunikasi Protokol: Menggunakan komunikasi stdio antara agen dan server MCP  
- Penemuan Fungsi: Menemukan fungsi yang tersedia secara otomatis dari server MCP  
- Streaming Respons: Menangkap dan mencatat panggilan fungsi secara real-time  
- Rendering HTML: Memformat respons agen dengan tabel bergaya dan tampilan interaktif  

Langkah Selanjutnya:  
- Integrasikan server MCP tambahan (cuaca, penerbangan, restoran)  
- Bangun sistem multi-agen yang menggabungkan protokol MCP dan A2A  
- Buat server MCP khusus untuk sumber data Anda sendiri  
- Terapkan memori percakapan yang persisten di seluruh sesi  
- Sebarkan agen ke Azure Functions dengan orkestrasi server MCP  
- Tambahkan autentikasi pengguna dan kemampuan pemesanan  

Keunggulan Utama Arsitektur MCP:  
- Standarisasi: Protokol universal untuk menghubungkan agen AI ke alat eksternal  
- Data Real-time: Akses ke informasi terkini dan langsung dari berbagai layanan  
- Ekstensibilitas: Integrasi mudah dengan sumber data dan alat baru  
- Interoperabilitas: Berfungsi di berbagai kerangka kerja AI dan platform agen  
- Pemisahan Tanggung Jawab: Pemisahan yang jelas antara logika AI dan akses data eksternal  



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diketahui bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan manusia profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
